In [4]:
# pip install datasets

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("wmt17", "de-en")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/5906184 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3004 [00:00<?, ? examples/s]

In [5]:
data = dataset["validation"]

In [6]:
data[2:5]

{'translation': [{'de': 'Die beiden wollten über die Umsetzung der internationalen Vereinbarung sowie über Teherans destabilisierende Maßnahmen im Nahen Osten sprechen.',
   'en': "The two wanted to talk about the implementation of the international agreement and about Teheran's destabilising activities in the Middle East."},
  {'de': 'Bei der Begegnung soll es aber auch um den Konflikt mit den Palästinensern und die diskutierte Zwei-Staaten-Lösung gehen.',
   'en': 'The meeting was also planned to cover the conflict with the Palestinians and the disputed two state solution.'},
  {'de': 'Das Verhältnis zwischen Obama und Netanyahu ist seit Jahren gespannt.',
   'en': 'Relations between Obama and Netanyahu have been strained for years.'}]}

In [7]:
#put english and german into seperate lists
input_sentences = []
reference_sentences = []

for example in data:
    english_translation = example["translation"]["en"]
    german_translation = example["translation"]["de"]
    input_sentences.append(english_translation)
    reference_sentences.append(german_translation)

In [8]:
data['translation'][2]['de']

'Die beiden wollten über die Umsetzung der internationalen Vereinbarung sowie über Teherans destabilisierende Maßnahmen im Nahen Osten sprechen.'

In [9]:
len(input_sentences)

2999

In [10]:
len(reference_sentences)

2999

In [12]:
# !pip install transformers
# !pip install sentencepiece

In [13]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm

In [ ]:
spm.__version__

'0.1.99'

In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load T5 model and tokenizer
model_name = "t5-small"  # You can use different T5 variants like "t5-base", "t5-large", etc.
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
# input_sentences = input_sentences[:3]
# reference_sentences = reference_sentences[:3]

# Translate sentences
translated_sentences = []

for sentence in input_sentences:
    input_ids = tokenizer.encode("translate English to German: " + sentence, return_tensors="pt",max_length=1024)

    with torch.no_grad():
        output = model.generate(input_ids,max_length=1024,num_beams=1, early_stopping = True,top_k=1)

    translated = tokenizer.decode(output[0], skip_special_tokens=True)
    translated_sentences.append(translated)

# # Print translated sentences
# for eng, ger in zip(input_sentences, translated_sentences):
#     print(f"English: {eng}")
#     print(f"German: {ger}\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [16]:
translated_sentences[90]

'Ban hat auf einer Nachrichtenkonferenz am Mittwoch gesagt, dass er vorhaben werde, mit Außenministern der fünf ständigen Ratsnationen - den USA, Russland, China, Großbritannien und Frankreich - auf der Seite der Ministersitzung der Generalversammlung im Laufe dieses Monats zu treffen, um Syrien zu diskutieren.'

In [17]:
reference_sentences[90]

'Ban sagte einer Pressekonferenz am Mittwoch, dass er plant, die Außenminister der fünf ständigen Mitglieds-Nationen des Rates - USA, Russland, China, Großbritannien und Frankreich - am Rande der Ministersitzung der Generalversammlung noch in diesem Monat zu treffen, um die Lage in Syrien zu diskutieren.'

In [18]:
source = [[sentence.split()] for sentence in input_sentences]
references = [[sentence.split()] for sentence in reference_sentences]
hypotheses = [sentence.split() for sentence in translated_sentences]

In [19]:
vocab_size = tokenizer.vocab_size
print(vocab_size)

32100


In [21]:
# !pip install --upgrade pip  # ensures that pip is current
# !pip install unbabel-comet

In [26]:
from comet import download_model, load_from_checkpoint

comt_model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comt_model_path)
data = [
    {
        "src": source,
        "mt":  hypotheses,
        "ref": references
    }
]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
  rank_zero_warn(


In [ ]:
# Calculate COMET score
comet_model_output = comet_model.predict(data, batch_size=8, gpus=1)
print(comet_model_output)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 1/1 [00:22<00:00, 22.41s/it]

Prediction([('scores', [0.9291549921035767]), ('system_score', 0.9291549921035767)])


In [23]:
# pip install sacrebleu

In [25]:
from sacrebleu.metrics import CHRF

# calculate the chrf
chrf = CHRF()
print(f'chrF2 score is :{chrf.corpus_score(translated_sentences, [reference_sentences])}')

chrF2 score is :chrF2 = 58.39


In [24]:
from sacrebleu.metrics import BLEU

# calculate the blue score
bleu = BLEU()

print(f'BLEU score is : {bleu.corpus_score(translated_sentences, [reference_sentences])}')

BLEU score is : BLEU = 31.05 62.3/37.1/24.6/16.9 (BP = 0.992 ratio = 0.992 hyp_len = 62155 ref_len = 62669)
